In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a = 5/3

In [ ]:
lambdas = a/np.array([0.08,0.10]) 
folder_path = r"H:\phd stuff\tidy3d\structures\08_13_2024 Luis H5 18.01a\sample 2"
project_name = "08_16_2024 STL RCP Sample Test Beam Spreading"
postprocess_results = []
runtime_ps = 40e-12
min_steps_per_lambda = 15
for direction in ["z"]: 
    sim_name = f"RCP  {a/lambdas[0]} - {a/lambdas[1]}"
    for filename in os.listdir(folder_path):
     for perm in [40]:
        print(filename)
        if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
            continue 
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction=direction, lambda_range=lambdas,
                                            box_size=18*a,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda, permittivity=perm,use_permittivity=True,
                                           scaling=1,shuoff_condtion=1e-20, verbose=True, 
                                           monitors=["flux"], freqs=200, 
                                           cut_condition=1.0, source="tight", tight_percentage=0.05, absorbers=150, width=0.2, sim_name=sim_name,
                                        multiplicate_size=True, multiplication_factor=4
                                           
                                           )
           

            


In [ ]:
sim = structure_1.sim
boundaries= td.BoundarySpec(
            x=td.Boundary(plus=td.Absorber(num_layers=200),minus=td.Absorber(num_layers=200)),
            y=td.Boundary(plus=td.Absorber(num_layers=200),minus=td.Absorber(num_layers=200)),
            z=td.Boundary(plus=td.Absorber(num_layers=200),minus=td.Absorber(num_layers=200)),
        )
Lx, Ly, Lz =structure_1.Lx+4,structure_1.Ly+4,structure_1.Lz+10


# slab= td.Structure(
#                     geometry=td.Box(
#                         center= (0,0,0),
#                         size=(
#                               structure_1.t_slab_x, 
#                               structure_1.t_slab_y, 
#                               structure_1.t_slab_z
#                               ),
#                     ),
#                     medium=structure_1.dielectric,
#                     name=f'slab{0}',
#                     )


time_monitorFieldLateral = td.FieldTimeMonitor(
                center = (0,0,0),
                size = (0,Ly,Lz),
                    start=0,
                    stop=structure_1.t_stop,
                    interval=200,
                    fields=["Ex", "Ey", "Ez"],
                    name="time_monitorFieldLateral",
                )



time_monitorT = td.FluxTimeMonitor(
                    center=[
                        (structure_1.Lx - structure_1.spacing)*0.5 if structure_1.direction == "x" else 0, 
                        (structure_1.Ly - structure_1.spacing)*0.5 if structure_1.direction == "y" else 0, 
                        (structure_1.Lz - structure_1.spacing)*0.5 if structure_1.direction == "z" else 0


                        ],
                    size=[
                        structure_1.t_slab_x,structure_1.t_slab_y,0
                        ],
                    interval = 200,
                    name="time_monitorT",

                )



sim = sim.copy(update={'boundary_spec':boundaries,'size':[Lx,Ly,Lz],"monitors":[time_monitorT,time_monitorFieldLateral]})





In [ ]:
# fig, ax = plt.subplots(1, tight_layout=True, figsize=(16, 8))
# freqs_plot = (structure_1.freq_range[0], structure_1.freq_range[1])
# sim.plot_eps(x=0, freq=freqs_plot[0], ax=ax)
# #sim.plot(x=0, ax=ax)
# plt.show()
# fig, ax = plt.subplots(1, tight_layout=True, figsize=(16, 8))
# freqs_plot = (structure_1.freq_range[0], structure_1.freq_range[1])
# sim.plot_eps(z=0, freq=freqs_plot[0], ax=ax)
# #sim.plot(z=0, ax=ax)
# plt.show()


In [ ]:
if True:

        id =web.upload(sim, folder_name=project_name,task_name=sim_name, verbose=True)
        web.start(task_id = id)
        web.monitor(task_id=id,verbose=True)
        ids = '\n' + id
        incidence_folder = "z_incidence"
        file_path = rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}/{sim_name}.txt"
        # Check if the folder exists
        if not os.path.exists( rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}"):
            os.makedirs(rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}")
            print(f"Folder '{project_name}/{incidence_folder}' created successfully.")

        # Open file in write mode
        with open(file_path, "w") as file:
            # Write the string to the file
            file.write(ids)